# 卷积格式说明
## 对[1, 1, H, W]的卷积实现
**卷积运算转换为：im2col矩阵 * kernel展开矩阵**  
**im2col行的数目：与kernel矩阵做运算的子矩阵的个数=im2colH = outputH * outputW**  
**im2col列的数目：kernel中元素的个数=im2colW=kernelH * kernelW**  
**kernel直接展开为列向量即可**

## 对[1, C, H, W]的卷积实现
**kernel维度：kernelO, kernelI, kernelH, kernelW**  
**im2col列的数目：kernel中元素的个数im2colW = kernelI * kernelH * kernelW**  
***这里必须注意展开后kernel的转置和矩阵乘法计算完成后的HWC到CHW的转换***

## 对[B, C, H, W]的卷积实现
**增加一个batch维度即可**  
**这里的im2col的做法是：行的数目乘以batchSize，即在行的后续增加batch的数据**

In [285]:
# [1, 1, H, W]的卷积实现
import numpy as np
import torch
from torch import nn
import torch.nn.functional  as F

inputH, inputW = 7, 7
kernelH, kernelW = 3, 3
input = np.arange(0, inputH * inputW).reshape(inputH, inputW).astype(np.int32)
kernel = np.arange(kernelH * kernelW, dtype=np.int32).reshape(kernelH, kernelW)
# print(input)
# print(kernel)

In [286]:
# 获取特征图的形状
inputH, inputW = input.shape
# 获取filter kernel的形状
kernelH, kernelW = kernel.shape
# 设置填充和步幅
padding = 0
stride = 1
# 计算卷积操作后输出特征图的行数和列数，也就是特征图的大小
outputH = int((inputH - kernelH + padding + stride) / stride)
outputW = int((inputW - kernelW + padding + stride) / stride)
# print(outputH, outputW)

# 计算卷积转换矩阵im2col的形状
im2colH = outputH * outputW
im2colW = kernelH * kernelW
im2colInput = np.zeros([im2colH, im2colW], dtype=np.int32)

# 将输入特征图input继续卷积展开
# i，j这两层循环遍历输出的高度和宽度，即在行和列这两个维度上与kernel矩阵分别运算的次数
# 同时im2col矩阵的行(也可以是列，是具体实现而定) = outputH * outputW
for i in range(outputH):
    for j in range(outputW):
        # i，j也是kernel在input移动时在input上的起始点索引
        # ii，jj是kernel维度上的索引
        for ii in range(kernelH):
            inputRowidx = i + ii       # input上的行索引
            for jj in range(kernelW):
                inputColidx = j + jj   # input上的列索引
                im2colInput[i * outputW + j][ii * kernelW + jj] = input[inputRowidx][inputColidx]
# print(im2colInput)
                
# 将卷积核展开
im2colKernel = kernel.reshape(kernelH * kernelW, -1)
# 计算卷积输出
output = np.matmul(im2colInput, im2colKernel).reshape(outputH, outputW)
print("my convolution implementation:")
print(output)

# pytorch验证计算结果正确性
inputTensor = input.reshape(1, 1, inputH, inputW)
weightTensor = kernel.reshape(1, 1, kernelH, kernelW)
inputTensor = torch.from_numpy(inputTensor)
weightTensor = torch.from_numpy(weightTensor)

output = F.conv2d(inputTensor, weightTensor, padding=padding, stride=stride)
output = output.reshape(outputH, outputW)
print("\ntorch convolution implementation:")
print(output)

my convolution implementation:
[[ 420  456  492  528  564]
 [ 672  708  744  780  816]
 [ 924  960  996 1032 1068]
 [1176 1212 1248 1284 1320]
 [1428 1464 1500 1536 1572]]

torch convolution implementation:
tensor([[ 420,  456,  492,  528,  564],
        [ 672,  708,  744,  780,  816],
        [ 924,  960,  996, 1032, 1068],
        [1176, 1212, 1248, 1284, 1320],
        [1428, 1464, 1500, 1536, 1572]], dtype=torch.int32)


In [287]:
# [1, C, H, W]的卷积实现
import numpy as np
import torch
from torch import nn
import torch.nn.functional  as F

intputC, inputH, inputW = 5, 7, 7
kernelO, kernelI, kernelH, kernelW = 3, 5, 3, 3
input = np.arange(0, intputC * inputH * inputW).reshape(intputC, inputH, inputW).astype(np.int32)
kernel = np.arange(kernelO * kernelI * kernelH * kernelW, dtype=np.int32)\
           .reshape(kernelO, kernelI, kernelH, kernelW)
# kernel = np.ones([kernelO, kernelI, kernelH, kernelW], dtype=np.int32)
# print(input)
# print(kernel)

In [288]:
# 获取特征图的形状
inputC, inputH, inputW = input.shape
# 获取filter kernel的形状
kernelO, kernelI, kernelH, kernelW = kernel.shape
# 设置填充和步幅
padding = 0
stride = 1
# 计算卷积操作后输出特征图的维度信息
outputC = kernelO
outputH = int((inputH - kernelH + padding + stride) / stride)
outputW = int((inputW - kernelW + padding + stride) / stride)
# print(outputC, outputH, outputW)

# 计算卷积转换矩阵im2col的形状
im2colH = outputH * outputW
im2colW = kernelI * kernelH * kernelW   # Notice: 与[1, 1, H, W]不同
im2colInput = np.zeros([im2colH, im2colW], dtype=np.int32)

# 将输入特征图input继续卷积展开
# i，j这两层循环遍历输出的高度和宽度，即在行和列这两个维度上与kernel矩阵分别运算的次数
# 同时im2col矩阵的行(也可以是列，是具体实现而定) = outputH * outputW
for i in range(outputH):
    for j in range(outputW):
        # i，j也是kernel在input移动时在input上的起始点索引
        # cc, ii，jj是kernel维度上的索引
        for cc in range(kernelI):          # channel维度
            for ii in range(kernelH):
                inputRowidx = i + ii       # input上的行索引
                for jj in range(kernelW):
                    inputColidx = j + jj   # input上的列索引
                    im2colInput[i * outputW + j][cc * kernelH * kernelW + ii * kernelW + jj] \
                        = input[cc][inputRowidx][inputColidx]
# print(im2colInput)
# print(im2colInput.shape)

# 将卷积核展开
im2colKernel = kernel.reshape(kernelO, -1).T    # Notice: 这里的转置是必须的
# print(im2colKernel)
# print(im2colKernel.shape)

# 计算卷积输出
# output = np.matmul(im2colInput, im2colKernel).reshape(outputC, outputH, outputW)  # Error!!!维度信息错误了！！！
output = np.matmul(im2colInput, im2colKernel).reshape(outputH, outputW, outputC)    # Pass, 计算完成后是HWC
output = np.transpose(output, (2, 0, 1))                                            # 需要transpose为CHW
print("my convolution implementation:")
print(output)

# pytorch验证计算结果正确性
inputTensor = input.reshape(1, inputC, inputH, inputW)
weightTensor = kernel.reshape(kernelO, kernelI, kernelH, kernelW)
inputTensor = torch.from_numpy(inputTensor)
weightTensor = torch.from_numpy(weightTensor)

output = F.conv2d(inputTensor, weightTensor, padding=padding, stride=stride)
output = output.reshape(outputC, outputH, outputW)
print("\ntorch convolution implementation:")
print(output)

my convolution implementation:
[[[145290 146280 147270 148260 149250]
  [152220 153210 154200 155190 156180]
  [159150 160140 161130 162120 163110]
  [166080 167070 168060 169050 170040]
  [173010 174000 174990 175980 176970]]

 [[359940 362955 365970 368985 372000]
  [381045 384060 387075 390090 393105]
  [402150 405165 408180 411195 414210]
  [423255 426270 429285 432300 435315]
  [444360 447375 450390 453405 456420]]

 [[574590 579630 584670 589710 594750]
  [609870 614910 619950 624990 630030]
  [645150 650190 655230 660270 665310]
  [680430 685470 690510 695550 700590]
  [715710 720750 725790 730830 735870]]]

torch convolution implementation:
tensor([[[145290, 146280, 147270, 148260, 149250],
         [152220, 153210, 154200, 155190, 156180],
         [159150, 160140, 161130, 162120, 163110],
         [166080, 167070, 168060, 169050, 170040],
         [173010, 174000, 174990, 175980, 176970]],

        [[359940, 362955, 365970, 368985, 372000],
         [381045, 384060, 387075, 3

In [289]:
# [B, C, H, W]的卷积实现
import numpy as np
import torch
from torch import nn
import torch.nn.functional  as F

inputB, intputC, inputH, inputW = 8, 5, 7, 7
kernelO, kernelI, kernelH, kernelW = 3, 5, 3, 3
input = np.arange(0, inputB * intputC * inputH * inputW).reshape(inputB, intputC, inputH, inputW).astype(np.int32)
kernel = np.arange(kernelO * kernelI * kernelH * kernelW, dtype=np.int32)\
           .reshape(kernelO, kernelI, kernelH, kernelW)
# kernel = np.ones([kernelO, kernelI, kernelH, kernelW], dtype=np.int32)
# print(input.shape)
# print(kernel)

In [290]:
# 获取特征图的形状
inputB, inputC, inputH, inputW = input.shape
# 获取filter kernel的形状
kernelO, kernelI, kernelH, kernelW = kernel.shape
# 设置填充和步幅
padding = 0
stride = 1
# 计算卷积操作后输出特征图的维度信息
outputB = inputB    # batch size
outputC = kernelO
outputH = int((inputH - kernelH + padding + stride) / stride)
outputW = int((inputW - kernelW + padding + stride) / stride)
# print(outputC, outputH, outputW)

# 计算卷积转换矩阵im2col的形状
im2colH = outputB * outputH * outputW   # Notice: 与[1, C, H, W]不同
im2colW = kernelI * kernelH * kernelW   # Notice: 与[1, 1, H, W]不同
im2colInput = np.zeros([im2colH, im2colW], dtype=np.int32)

# 将输入特征图input继续卷积展开
# i，j这两层循环遍历输出的高度和宽度，即在行和列这两个维度上与kernel矩阵分别运算的次数
# 同时im2col矩阵的行(也可以是列，是具体实现而定) = outputH * outputW
for b in range(outputB):                        # batch维度上的loop
    for i in range(outputH):
        for j in range(outputW):
            # i，j也是kernel在input移动时在input上的起始点索引
            # cc, ii，jj是kernel维度上的索引
            for cc in range(kernelI):           # channel维度
                for ii in range(kernelH):
                    inputRowidx = i + ii        # input上的行索引
                    for jj in range(kernelW):
                        inputColidx = j + jj    # input上的列索引
                        im2colInput[b * outputH * outputW + i * outputW + j][cc * kernelH * kernelW + ii * kernelW + jj] \
                            = input[b][cc][inputRowidx][inputColidx]
# print(im2colInput)
print("im2colInput.shape: ", im2colInput.shape)

# 将卷积核展开
im2colKernel = kernel.reshape(kernelO, -1).T    # Notice: 这里的转置是必须的
# print(im2colKernel)
print("im2colKernel.shape: ", im2colKernel.shape)

# 计算卷积输出
output = np.matmul(im2colInput, im2colKernel).reshape(outputB, outputH, outputW, outputC)
output = np.transpose(output, (0, 3, 1, 2))     # 需要transpose: BHWC转换到BCHW
# print("my convolution implementation:")
# print(output)
print("output.shape: ", output.shape)

# pytorch验证计算结果正确性
inputTensor = input.reshape(inputB, inputC, inputH, inputW)
weightTensor = kernel.reshape(kernelO, kernelI, kernelH, kernelW)
inputTensor = torch.from_numpy(inputTensor)
weightTensor = torch.from_numpy(weightTensor)

pyoutput = F.conv2d(inputTensor, weightTensor, padding=padding, stride=stride)
# print("\ntorch convolution implementation:")
# print(pyoutput)
print("pyoutput.shape: ", pyoutput.shape)

# Convert tensor to ndarray
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
np.testing.assert_allclose(to_numpy(pyoutput), output, rtol=1e-03, atol=1e-05)
print("PASS!")

im2colInput.shape:  (200, 45)
im2colKernel.shape:  (45, 3)
output.shape:  (8, 3, 5, 5)
pyoutput.shape:  torch.Size([8, 3, 5, 5])
PASS!
